# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,carutapera,-1.1950,-46.0200,26.07,80,11,3.93,BR,1723170346
1,1,kapa'a,22.0752,-159.3190,29.18,66,0,5.66,US,1723170929
2,2,papao,-17.7500,-149.5500,24.08,90,72,6.16,PF,1723170930
3,3,chonchi,-42.6128,-73.8092,6.41,95,100,1.43,CL,1723170657
4,4,nantucket,41.2835,-70.0995,17.06,100,100,4.63,US,1723170932


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [3]:
%%capture --no-display

# Configure the map plot
map_plot_humidity = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    frame_width = 800,
    frame_height = 600,
    tiles="OSM",
    size="Humidity",
    color="City"

)

# Display the map
map_plot_humidity

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
# Weather Criteria: Max Temperature between 21 an 27 degrees, Wind Speed less than 4.5 m/s, and zero cloudiness
ideal_city_df = city_data_df[(city_data_df["Max Temp"] <= 27.00) & (city_data_df["Max Temp"] >= 21) & (city_data_df["Wind Speed"] < 4.5) & (city_data_df["Cloudiness"]==0)]

# Drop any rows with null values
ideal_city_df.dropna()

# Display sample data
ideal_city_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
95,95,suez,29.9737,32.5263,25.28,70,0,3.17,EG,1723171052
233,233,busayra,30.7376,35.6078,21.32,40,0,1.77,JO,1723171221
244,244,protaras,35.0125,34.0583,25.55,94,0,0.45,CY,1723171236
269,269,mancio lima,-7.6142,-72.8958,25.02,57,0,0.60,BR,1723171265
273,273,cortez,37.3489,-108.5859,23.42,38,0,3.60,US,1723171271
288,288,cua,10.1589,-66.8858,21.79,96,0,1.41,VE,1723171288
293,293,springfield,37.2153,-93.2982,23.11,84,0,2.24,US,1723170606
302,302,la puebla de montalban,39.8630,-4.3592,25.66,25,0,1.71,ES,1723171305
315,315,shahr-e babak,30.1165,55.1186,26.42,42,0,3.86,IR,1723171322
320,320,itaituba,-4.2761,-55.9836,23.90,47,0,0.17,BR,1723171327


### Step 3: Create a new DataFrame called `hotel_df`.

In [22]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_city_df[["City","Country","Lat","Lng","Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"]=""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
95,suez,EG,29.9737,32.5263,70,
233,busayra,JO,30.7376,35.6078,40,
244,protaras,CY,35.0125,34.0583,94,
269,mancio lima,BR,-7.6142,-72.8958,57,
273,cortez,US,37.3489,-108.5859,38,
288,cua,VE,10.1589,-66.8858,96,
293,springfield,US,37.2153,-93.2982,84,
302,la puebla de montalban,ES,39.8630,-4.3592,25,
315,shahr-e babak,IR,30.1165,55.1186,42,
320,itaituba,BR,-4.2761,-55.9836,47,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [40]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "limit":20,
    "categories":"accommodation.hotel",
    "apiKey":geoapify_key
}
# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index,"Lat"]
    longitude = hotel_df.loc[index,"Lng"]
    
    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
suez - nearest hotel: فندق لوسيندا
busayra - nearest hotel: Dana Nabil Eco Camp House
protaras - nearest hotel: Capo Bay
mancio lima - nearest hotel: No hotel found
cortez - nearest hotel: BEST WESTERN Turquoise Inn & Suites
cua - nearest hotel: No hotel found
springfield - nearest hotel: Hotel Vandivort
la puebla de montalban - nearest hotel: Hostal Dorado
shahr-e babak - nearest hotel: هتل شهرداری
itaituba - nearest hotel: TG Hotel
morgan hill - nearest hotel: La Quinta Inn & Suites Morgan Hill
al bawiti - nearest hotel: Old Oasis Hotel
georgiyevka - nearest hotel: Диана гостиница
conceicao do araguaia - nearest hotel: No hotel found
ben mehidi - nearest hotel: No hotel found
prado - nearest hotel: Pousada Recanto do Prado
cide - nearest hotel: No hotel found
ghadamis - nearest hotel: فندق عين الفارس
alice springs - nearest hotel: Aurora Alice Springs
remire-montjoly - nearest hotel: Complexe Belova
milazzo - nearest hotel: La Chicca
matoury - nearest hotel: La 

,City,Country,Lat,Lng,Humidity,Hotel Name
95,suez,EG,29.9737,32.5263,70,فندق لوسيندا
233,busayra,JO,30.7376,35.6078,40,Dana Nabil Eco Camp House
244,protaras,CY,35.0125,34.0583,94,Capo Bay
269,mancio lima,BR,-7.6142,-72.8958,57,No hotel found
273,cortez,US,37.3489,-108.5859,38,BEST WESTERN Turquoise Inn & Suites
288,cua,VE,10.1589,-66.8858,96,No hotel found
293,springfield,US,37.2153,-93.2982,84,Hotel Vandivort
302,la puebla de montalban,ES,39.8630,-4.3592,25,Hostal Dorado
315,shahr-e babak,IR,30.1165,55.1186,42,هتل شهرداری
320,itaituba,BR,-4.2761,-55.9836,47,TG Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [41]:
%%capture --no-display

# Configure the map plot
hotel_map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo=True,
    frame_width = 800,
    frame_height = 600,
    tiles="OSM",
    size="Humidity",
    color="City",
    hover_cols=["Hotel Name","Country"]
)

# Display the map
hotel_map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)